# Kelvin-Helmholtz Instability with SPH

Kelvin-Helmholtz instabilities are one of the classic test of hydrodynamics solvers (despite not having an analytic solution, and thus not really being a rigorous test... they look great though!). Even so, as you can see below they appear very readily in nature!

 <p align="center">
  <img src="cloud_kh.png" width="600">
</p>

In this notebook, we will use Kelvin-Helmholtz instabilities to demonstrate the performance and characteristics of different SPH schemes available in SWIFT.

Before we get stuck in though:

## What is a Kelvin-Helmholtz Instability?

Kelvin-Helmholtz instabilities arise from the shear flow between two fluid layers moving at different velocities. Small perturbations at the interface grow exponentially, leading to the formation of vortical structures. 

The growth rate of the instability depends on:
- The velocity difference between the layers
- The density contrast between the fluids
- The wavelength of the perturbation

## What we'll do in this notebook

1. **Generate initial conditions** with two fluid layers at different velocities
2. **Create parameter files** for SWIFT
3. **Run the simulation** (separately from the notebook)
4. **Visualize the results** and create movies
5. **Explore different solvers/parameters** to see how they affect the instability growth

Let's get started!

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
import h5py
import unyt
from scipy.spatial.distance import cdist
import os
import subprocess
from swiftsimio import Writer
from swiftsimio.units import cosmo_units

# Set up plotting
plt.style.use('default')
plt.rcParams['figure.figsize'] = [10, 8]
plt.rcParams['font.size'] = 12

# Define some constants we need 
scheme = "gadget2"

## 1. Setting up the Initial Conditions

For the Kelvin-Helmholtz instability, we need to create two layers of fluid with different velocities. The function below sets this up for us based on the input arguments. By all means explore different inputs than those demonstrated below but:

- The more particles the longer the simulation will take. We only have limited resources for each student and a limited amount of time for the workshop. In short, don't go too high! But in the same breathe, more particles will give a better result.
- This comes with no warranty. I'm sure its entirely possible to very readily create a set of ICs which may be malformed in some way.

That all said, review the code below and make sure you understand what it is doing. Then run the function with arguments of your choice to generate a set of initial conditions.

In [ ]:
import numpy as np
from swiftsimio.visualisation import generate_smoothing_lengths

def generate_kelvin_helmholtz_ic(
    sqrt_n_part,
    velocity_shear=1.0,           # total ΔV between center and outskirts
    density_contrast=2.0,         # rho_center / rho_outskirts
    perturbation_amplitude=0.1,   # amplitude of KH seed in v_y (SWIFT example: 0.1)
    gamma=5.0/3.0,
    pressure=2.5,                 # common pressure P1=P2 for equilibrium
    band_halfwidth_frac=0.25,     # central band: |y - Ly/2| < band_halfwidth_frac * Ly
    mode_number=2,                # sin(2π * mode * x / Lx); SWIFT uses 2 → sin(4πx/Lx)
    z_thickness=0.1,              # thin slab thickness for 2D runs (Header.Dimension=2)
    target_neighbours=30          # typical for 2D SPH; (use ~64 for 3D)
):
    """
    Generate Kelvin–Helmholtz ICs (2D, SWIFT-style) with smoothing lengths
    computed by swiftsimio.generate_smoothing_lengths.

    Returns a dict with arrays ready to write to a SWIFT IC HDF5:
      positions (N,3), velocities (N,3), masses (N,),
      internal_energies (N,), smoothing_length (N,), ids (N,),
      n_particles, box_size (3,)
    """
    # The boxsize is a fixed unit square 
    Lx, Ly = 1.0, 1.0

    # --- densities (dimensionless, relative) ---
    rho_out = 1.0
    rho_ctr = density_contrast * rho_out

    # --- grid resolutions (two independent uniform grids like the example) ---
    L2 = int(sqrt_n_part)                               # outskirts grid per edge
    num2 = L2 * L2
    L1 = int(np.sqrt(num2 * (rho_ctr / rho_out)))       # center grid per edge to match number-density ratio
    num1 = L1 * L1

    # --- helper to make a centered grid on [0,Lx]x[0,Ly] ---
    def make_grid(L):
        xs = (np.arange(L) + 0.5) / L
        ys = (np.arange(L) + 0.5) / L
        xx, yy = np.meshgrid(xs, ys, indexing="xy")
        coords = np.zeros((L * L, 3), dtype=float)
        coords[:, 0] = xx.ravel() * Lx
        coords[:, 1] = yy.ravel() * Ly
        coords[:, 2] = 0.5 * z_thickness
        return coords

    coords1 = make_grid(L1)    # central band candidates
    coords2 = make_grid(L2)    # outskirts candidates

    # --- pressure equilibrium: u = P/[(γ-1)ρ] ---
    u1_val = pressure / ((gamma - 1.0) * rho_ctr)
    u2_val = pressure / ((gamma - 1.0) * rho_out)
    u1 = np.full(num1, u1_val, dtype=float)
    u2 = np.full(num2, u2_val, dtype=float)

    # --- base shear velocities: +ΔV/2 in center, -ΔV/2 in outskirts ---
    v1 = +0.5 * velocity_shear
    v2 = -0.5 * velocity_shear
    vel1 = np.zeros((num1, 3), dtype=float); vel1[:, 0] = v1
    vel2 = np.zeros((num2, 3), dtype=float); vel2[:, 0] = v2

    # --- select central band & outskirts complement (like the example) ---
    mid_y = 0.5 * Ly
    band_half = band_halfwidth_frac * Ly
    sel1 = np.abs(coords1[:, 1] - mid_y) < band_half
    sel2 = np.abs(coords2[:, 1] - mid_y) > band_half

    coords = np.concatenate([coords1[sel1], coords2[sel2]], axis=0)
    vel    = np.concatenate([vel1[sel1],   vel2[sel2]],   axis=0)
    u      = np.concatenate([u1[sel1],     u2[sel2]],     axis=0)

    N = coords.shape[0]
    ids = np.arange(1, N + 1, dtype=np.int64)

    # --- constant particle mass (density contrast via number density) ---
    area = Lx * Ly
    f_center = 2.0 * band_halfwidth_frac                      # area fraction
    rho_mean = f_center * rho_ctr + (1.0 - f_center) * rho_out
    m_val = rho_mean * area / float(N)
    masses = np.full(N, m_val, dtype=float)

    # --- KH seed: two Gaussian sheets at y=0.25Ly and 0.75Ly, sinusoidal in x ---
    kx = 2.0 * np.pi * mode_number / Lx
    sigma = 0.05 * Ly / np.sqrt(2.0)                          # scale with Ly (as in the example)
    x = coords[:, 0]; y = coords[:, 1]
    vy_seed = (
        np.exp(-((y - 0.25 * Ly) ** 2) / (2.0 * sigma ** 2)) +
        np.exp(-((y - 0.75 * Ly) ** 2) / (2.0 * sigma ** 2))
    ) * np.sin(kx * x)
    vel[:, 1] += perturbation_amplitude * vy_seed

    return {
        "positions": coords,                 # (N,3)
        "velocities": vel,                   # (N,3)
        "masses": masses,                    # (N,)
        "internal_energies": u,              # (N,)
        "ids": ids,                          # (N,)
        "n_particles": N,
        "box_size": np.array([Lx, Ly, z_thickness], dtype=float),
    }


# Generate the initial conditions
ic_data = generate_kelvin_helmholtz_ic(200,
                                      velocity_shear=2.0,
                                      density_contrast=2.0,
                                      perturbation_amplitude=0.1)

print(f"Box size: {ic_data['box_size']}")
print(f"Number of particles: {ic_data['n_particles']}")

## 2. Visualizing the Initial Conditions

With the initial conditions generated, we can visualize them to ensure they look as expected. The cell below will create a set of subplots showing the density and velocity fields of the two fluid layers. 

In [ ]:
# Plot initial conditions
fig, axes = plt.subplots(1, 2, figsize=(12,7))

pos = ic_data['positions']
vel = ic_data['velocities']

# X-velocity field
im2 = axes[0].scatter(pos[:,0], pos[:,1], c=vel[:,0], s=1, cmap='RdBu_r')
axes[0].set_title('X-Velocity')
axes[0].set_xlabel('x')
axes[0].set_ylabel('y')
axes[0].set_aspect('equal')
cbar = plt.colorbar(im2, ax=axes[0], orientation='horizontal')
cbar.set_label('$x$ Velocity')

# Y-velocity field (perturbations)
im3 = axes[1].scatter(pos[:,0], pos[:,1], c=vel[:,1], s=1, cmap='RdBu_r')
axes[1].set_title('Y-Velocity (Perturbations)')
axes[1].set_xlabel('x')
axes[1].set_ylabel('y')
axes[1].set_aspect('equal')
cbar = plt.colorbar(im3, ax=axes[1], orientation='horizontal')
cbar.set_label('$y$ Velocity')

plt.tight_layout()
plt.show()

## 3. Writing Initial Conditions to HDF5 File

Before we can run the simulation, we need to write the initial conditions to the HDF5 file that SWIFT ingests. To do this we will use ``swiftsimio``, for more details see the [swiftsimio documentation](https://swiftsimio.readthedocs.io/en/latest/).  

In [ ]:
def write_swift_ic_file(ic_data, filename):
    """
    Write initial conditions to SWIFT-compatible HDF5 file
    """
    
    # Ensure output directory exists
    os.makedirs('../ics', exist_ok=True)
    filepath = f'../ics/{filename}'

    # Create the IC writer
    writer = Writer("cgs", boxsize=ic_data['box_size'] * unyt.cm, dimension=2)

    # Attach all the data we extracted
    writer.gas.coordinates = ic_data['positions'] * unyt.cm
    writer.gas.velocities = ic_data['velocities'] * unyt.cm / unyt.s
    writer.gas.masses = ic_data['masses'] * unyt.g
    writer.gas.internal_energy = ic_data['internal_energies'] * (unyt.cm / unyt.s)**2

    # Generate initial guess for smoothing lengths based on MIPS
    writer.gas.generate_smoothing_lengths(boxsize=ic_data['box_size'] * unyt.cm, dimension=2)

    # Write it
    writer.write(filepath)
        
    print(f"Written initial conditions to {filepath}")
    return filepath

# Write the IC file
ic_filename = f"kelvin_helmholtz_{scheme}.hdf5"
ic_filepath = write_swift_ic_file(ic_data, ic_filename)

We're nearly there now, just need a way to tell SWIFT what to do and SWIFT itself to execute. 

## 4. Creating Parameter File for SWIFT

We control SWIFT behaviour (i.e. what physics to include, what parameter values to run with and what files to ingest/output), via a yaml file. The cell below contains a simple template with the parameters we will need to run our Kelvin-Helmholtz simulation. 

Here we will start with a viscosity of 0.0, i.e. no artificial viscosity, to show how the instability fails to develop properly. we'll then come back later and re-run with a viscosity of 0.8 to see how it improves things.

Note that this template barely scratches the surface of what parameters can be used. You'll see more in the isolated galaxy simulation if you want to explore further.

In [ ]:
def create_swift_parameter_file(ic_filename, snap_name='kelvin_helmholtz', visc_alpha=0.8, time_end=1.0):
    """
    Create a SWIFT parameter file for the Kelvin-Helmholtz simulation
    """
    
    # Ensure output directory exists
    os.makedirs('../params', exist_ok=True)
    param_file = f'../params/{snap_name}.yml'

    # Ensure the snapshots directory exists
    os.makedirs(f'snapshots/{snap_name}', exist_ok=True)
    
    param_content = f"""
# Define the system of units to use internally. 
InternalUnitSystem:
  UnitMass_in_cgs:     1   # g 
  UnitLength_in_cgs:   1   # cm
  UnitVelocity_in_cgs: 1   # cm/s  
  UnitCurrent_in_cgs:  1   # amperes
  UnitTemp_in_cgs:     1   # kelvin

# Parameters governing the time integration
TimeIntegration:
  time_begin: 0.0    # The starting time of the simulation (in internal units).
  time_end:   {time_end}   # The end time of the simulation (in internal units).
  dt_min:     1e-6  # The minimal time-step size of the simulation (in internal units).
  dt_max:     1e-2  # The maximal time-step size of the simulation (in internal units).


# Parameters governing the snapshots
Snapshots:
  basename:            snapshots/{snap_name}/{snap_name}  # Common part of the name of output files
  time_first:          0.               # Time of the first output (in internal units)
  delta_time:          0.01      # Time difference between consecutive outputs (in internal units)
  compression:         1              # GZIP compression level (0-9)

# Parameters governing the conserved quantities statistics
Statistics:
  delta_time:          0.01    # Time between statistics output

# Parameters for the hydrodynamics scheme
SPH:
  resolution_eta:        1.2348   # Target smoothing length in units of the mean inter-particle separation
  CFL_condition:         0.1      # Courant-Friedrichs-Levy condition for time integration
  viscosity_alpha:       {visc_alpha}      # (Optional) Override for the initial value of the artificial viscosity. In schemes that have a fixed AV, this remains as alpha throughout the run.


# Parameters related to the initial conditions
InitialConditions:
  file_name:  ../ics/{ic_filename}    # The file to read
  periodic:   1                       # Periodic boundary conditions
"""
    
    with open(param_file, 'w') as f:
        f.write(param_content)
    
    print(f"Created parameter file: {param_file}")
    return param_file

# Define viscosity parameter 
alpha_visc = 0.0

# Create parameter file
param_file = create_swift_parameter_file(ic_filename, snap_name=f"kelvin_helmholtz_{scheme}_{str(alpha_visc).replace(".", "p")}", visc_alpha=alpha_visc, time_end=2.0)

## 5. Configuring and compiling SWIFT 

To run the simulation we need to configure and compile SWIFT to run a 2D hydrodynamics simulation. This is done in the cell below (but assumes you have run the setup script that defines the relevant environment variables).

NOTE: You don't have to rerun this cell for every variation. 


In [ ]:
%%bash
module load intel_comp/2024.2.0 compiler-rt tbb compiler ucx/1.17.0 hdf5
cd ../swiftsim
./configure \
  --with-hydro=gadget2 \
  --with-hydro-dimension=2 \
  --enable-ipo \
  --with-kernel=wendland-C2 \
  --disable-mpi \
  --disable-hand-vec
make -j


## 6. Running the Simulation

Now we are ready to run SWIFT, we have the configured executable, initial conditions and parameter file. We'll run here with a SLURM submission script to submit the job to the DINE2 queue. This is so we can run with a suitable amount of resource without overloading the shared system. You may sit in the queue for a short while but the simulations should run quickly.

In [ ]:
def generate_slurm_script(param_file, job_name='swift-img', time_limit='00:10:00', nthreads=8):
    """
    Generate a SLURM submission script to run SWIFT with the given parameter file.
    """
    
    # Ensure output directory exists
    script_file = "submit"
    
    script_content = f"""#!/bin/bash -l
#SBATCH --ntasks=1
#SBATCH -J {job_name}
#SBATCH --output=log_kh.out
#SBATCH -p dine2
#SBATCH -A do020
#SBATCH --cpus-per-task={nthreads}
#SBATCH --time={time_limit}

module purge
module load intel_comp/2024.2.0 compiler-rt tbb compiler mpi ucx/1.17.0 parallel_hdf5/1.14.4 fftw/3.3.10 parmetis/4.0.3-64bit gsl/2.8

../swiftsim/swift --hydro --threads=$SLURM_CPUS_PER_TASK {param_file}

echo "Job done, info follows..."
sacct -j $SLURM_JOBID --format=JobID,JobName,Partition,AveRSS,MaxRSS,AveVMSize,MaxVMSize,Elapsed,ExitCode
exit

"""

    with open(script_file, 'w') as f:
        f.write(script_content)

    print(f"SLURM submission script generated: {script_file}")

generate_slurm_script(param_file, job_name=f"kh_{scheme}", time_limit='00:10:00', nthreads=8)

Now we just submit the job to the queue and wait.

In [ ]:
%%bash 
sbatch submit

If you want to check on the run you can use the `squeue --m` command to see the status of your jobs. Don't move on to the next cell before your job has finished! 

You can also look at the tail of the output file to see how things are progressing with `tail -100 log_kh.out` (or whatever you named your log above). 

In [ ]:
%%bash
squeue --m

# Visualise the simulation

With the simulation run we can now visualise the results using the cell below.

Once the video has been made, you can download it to your local machine using `rsync, e.g.

```bash
rsync -avz -P <user>@login8.cosma.dur.ac.uk:/path/to/movie.mp4 /path/to/local/directory/
``` 

In [ ]:
import glob
from matplotlib.animation import FuncAnimation
from tqdm import tqdm
from swiftsimio import load
from swiftsimio.visualisation import project_gas
from matplotlib.colors import LogNorm
import matplotlib

matplotlib.use("Agg")

# Make the videos directory if it doesn't exist
os.makedirs('videos', exist_ok=True)


def load_and_extract(filename):
    """
    Load the data and extract relevant info.
    """

    return load(filename)


def make_plot(filename, array, nx, ny, dx, dy):
    """
    Plot this frame. 

    Load the data and plop it on the grid using nearest
    neighbour searching for finding the 'correct' value of
    the density.
    """

    data = load_and_extract(filename)

    mesh = project_gas(data, nx).T.value

    array.set_array(mesh)

    return (array,)


def frame(n, *args):
    """
    Make a single frame. Requires the global variables plot and dpi.
    """

    global plot, dpi

    return make_plot(snapshot_files[n], plot, dpi, dpi, (0, 1), (0, 1))


# Look for snapshot files matching the current run
snapshot_files = sorted(glob.glob(f'snapshots/kelvin_helmholtz_{scheme}_{str(alpha_visc).replace(".", "p")}/kelvin_helmholtz_{scheme}_{str(alpha_visc).replace(".", "p")}_*.hdf5'))
print(f"Found {len(snapshot_files)} snapshot files.")

# Define the resolution of the output movie
dpi = 512

# Create a progress bar for feedback
frames = tqdm(np.arange(0, len(snapshot_files)), desc="Making frames")

# Set up the plot itself
fig, ax = plt.subplots(1, 1, figsize=(1, 1), frameon=False)
ax.axis("off")  # Remove annoying black frame.

# Load the first frame to get the normalisation
data = load_and_extract(snapshot_files[0])

# Create the first frame
mesh = project_gas(data, dpi).T.value

# Get the normalising percentiles from the first frame 
vmin = np.percentile(mesh, 5)
vmax = np.percentile(mesh, 99)

# Make the plot with the first frame so we can update it during the animation
plot = ax.imshow(
    mesh,
    extent=[0, 1, 0, 1],
    animated=True,
    interpolation="none",
    cmap="RdBu_r",
    norm=LogNorm(vmin=vmin, vmax=vmax, clip=True)
)

# Setup the  animation
anim = FuncAnimation(fig, frame, frames, interval=40, blit=False)

# Remove all whitespace
fig.subplots_adjust(left=0, bottom=0, right=1, top=1, wspace=None, hspace=None)

# Actually make the movie
anim.save(f"videos/kh_{scheme}_{str(alpha_visc).replace(".", "p")}.gif", dpi=dpi)


## Variations

### Including artificial viscosity 

Now, head back to the parameter file and change the viscosity from 0.0 to 0.8 and re-run the simulation. How does this change the results? 



### Different SPH schemes 

SWIFT has a range of different SPH schemes implemented. Below you can reconfigure SWIFT to use a different scheme and see how this changes the results. Just run one of the cells below and rerun the parameter file creation, submission script, submit it and then visualise the results. 

The cells below do not representan exhaustive list of the schemes available, just a few to try out. Also, there is no need to run each of them!

#### SPHENIX 

(The default)

In [ ]:
%%bash
module load intel_comp/2024.2.0 compiler-rt tbb compiler ucx/1.17.0 hdf5
cd ../swiftsim
./configure \
  --with-hydro=sphenix \
  --with-hydro-dimension=2 \
  --enable-ipo \
  --with-kernel=wendland-C2 \
  --disable-mpi
make -j

In [ ]:
scheme = "sphenix"

#### PHANTOM

In [ ]:
%%bash
module load intel_comp/2024.2.0 compiler-rt tbb compiler ucx/1.17.0 hdf5 
cd ../swiftsim
./configure \
  --with-hydro=phantom \
  --with-hydro-dimension=2 \
  --enable-ipo \
  --with-kernel=wendland-C2 \
  --disable-mpi
make -j

In [ ]:
scheme = "phantom"

#### Anarchy 

In [ ]:
%%bash
module load intel_comp/2024.2.0 compiler-rt tbb compiler ucx/1.17.0 hdf5 
cd ../swiftsim
./configure \
  --with-hydro=anarchy-pu \
  --with-hydro-dimension=2 \
  --enable-ipo \
  --with-kernel=wendland-C2 \
  --disable-mpi
make -j

In [ ]:
scheme = "anarchy"

By all means move on to the image simulation now or explore other schemes/parameters.